## Cherry picking protocol to run in the OT2 jupyter notebook

- make choosing labware and locations interactive
- import only requires xls file(s) from plate reader
- plots histogram and allows choice of threshold

In [ ]:
# imports
import opentrons.execute
from opentrons import simulate
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np


Set the options for the protocol below, then run the next two cells to take a look at your cherries.

In [ ]:
# protocol options
TESTING = True
CHERRIES_XLS = "cherry_pick_test.xls" # this should be the name of the excel file in the same directory as this script
CHERRY_PICK_AMOUNT = 5 # uL
FORMAT = 384 # 96 or 384

In [ ]:
# in here we should analyze the excel file and determine the cherries that we need to pick

# from screentools: this will only work for 384 well plates
def importPlates(
    xls_path: Str,
    plate_list: list,
    peptide_list: list,
    plate_format: int = 384,
    positive_controls: list = ["A1", "B1", "C1"],
    negative_controls: list = ["D1", "E1", "F1"],
):
    """
    If 384, import plate from single excel sheet, if 96, import an excel file with individual plates as sheets.
    """
    print("importing: ", xls_path)
    if plate_format == 384:
        # upper_left_location = (50, 2) # these should be right if the plate reader saved correctly
        upper_left_location = (52, 2) # these should be right if the plate reader saved correctly
        lower_right_location = (67, 25)

        df = pd.read_excel(xls_path)
        plate_df = df.iloc[
            upper_left_location[0] - 1 : lower_right_location[0],
            upper_left_location[1] : lower_right_location[1] + 1,
        ] # this is the dataframe of the plate with the shape (16, 24)

        plates = parsePlate384(plate_df, plate_list, peptide_list, positive_controls=positive_controls, negative_controls=negative_controls) # this should reshape the plate into a list of 384 wells

        return plates

    elif plate_format == 96:
        upper_left_location = (42, 2)
        lower_right_location = (49, 13)
        plates = []

        for i, plate_name in enumerate(plate_list):
            df = pd.read_excel(xls_path, sheet_name=i)
            plate_df = df.iloc[
                upper_left_location[0] - 1 : lower_right_location[0],
                upper_left_location[1] : lower_right_location[1] + 1,
            ]

            plate = parsePlate(plate_df)
            plate["plate_number"] = plate_name
            plate["peptide"] = peptide_list[i]

            plates.append(plate)

        return pd.concat(plates)

def parsePlate384(plate_df, plate_list, peptide_list, positive_controls=['A1', 'B1', 'C1'], negative_controls=['D1', 'E1', 'F1']):
    """
    Parses a 384 well plate. The first 12 columns are designated as plate 1 and
    the last 12 columns are designated as plate 2. Alternating rows are designated
    as substrate 1 and 2 respectively.
    """
    # get the first 12 columns of plate_df. These contain plate 1.
    plate_1 = plate_df.iloc[:, :12]
    # get the last 12 columns of plate_df. These contain plate 2.
    plate_2 = plate_df.iloc[:, 12:]
    # get the first set of alternating rows of plate_1. These contain substrate 1.
    plate_1_sub1 = plate_1.iloc[::2, :]
    # get the second set of alternating rows of plate_1. These contain substrate 2.
    plate_1_sub2 = plate_1.iloc[1::2, :]
    # get the first set of alternating rows of plate_2. These contain substrate 1.
    plate_2_sub1 = plate_2.iloc[::2, :]
    # get the second set of alternating rows of plate_2. These contain substrate 2.
    plate_2_sub2 = plate_2.iloc[1::2, :]

    # parse each plate as a 96 well plate
    plate_1_sub1 = parsePlate(plate_1_sub1)
    plate_1_sub2 = parsePlate(plate_1_sub2)
    plate_2_sub1 = parsePlate(plate_2_sub1)
    plate_2_sub2 = parsePlate(plate_2_sub2)

    # assign control wells to each plate
    plate_1_sub1 = assignControls(plate_1_sub1, positive=positive_controls, negative=negative_controls)
    plate_1_sub2 = assignControls(plate_1_sub2, positive=positive_controls, negative=negative_controls)
    plate_2_sub1 = assignControls(plate_2_sub1, positive=positive_controls, negative=negative_controls)
    plate_2_sub2 = assignControls(plate_2_sub2, positive=positive_controls, negative=negative_controls)

    # add the plate number and peptide to each plate
    plate_1_sub1["plate_number"] = plate_list[0]
    plate_1_sub1["peptide"] = peptide_list[0]
    plate_1_sub2["plate_number"] = plate_list[0]
    plate_1_sub2["peptide"] = peptide_list[1]
    plate_2_sub1["plate_number"] = plate_list[1]
    plate_2_sub1["peptide"] = peptide_list[0]
    plate_2_sub2["plate_number"] = plate_list[1]
    plate_2_sub2["peptide"] = peptide_list[1]
    

    # # now expand the plates into a list of wells
    # plate_1_sub1 = expandPlate(plate_1_sub1, plate_list[0], peptide_list[0])
    # plate_1_sub2 = expandPlate(plate_1_sub2, plate_list[0], peptide_list[1])
    # plate_2_sub1 = expandPlate(plate_2_sub1, plate_list[1], peptide_list[0])
    # plate_2_sub2 = expandPlate(plate_2_sub2, plate_list[1], peptide_list[1])

    # concatenate the plates into a single dataframe
    plate_df = pd.concat([plate_1_sub1, plate_1_sub2, plate_2_sub1, plate_2_sub2])

    return plate_df
    
cherries = importPlates(CHERRIES_XLS, ["A"], ["peptide1"], plate_format=FORMAT)

In [3]:
if TESTING:
    protocol = simulate.get_protocol_api('2.12')
else:
    protocol = opentrons.execute.get_protocol_api('2.12')

# Error to raise if there are setup issues.
class SetupError(Exception):
    pass

# labware
# first load cherry plate
if FORMAT == 96:
    # initially this will be unsupported
    raise SetupError("96 well plates are not yet supported")
elif FORMAT == 384:
    cherry_plate = protocol.load_labware('grenierbioone_384_wellplate_138ul', 1)

# load cherry destinations

/Users/rathbunc/.opentrons/robot_settings.json not found. Loading defaults
/Users/rathbunc/.opentrons/deck_calibration.json not found. Loading defaults
